In [1]:
import pandas as pd 
import requests
import os
import json
import numpy as np

In [2]:
with open(r'C:\Users\akobe\OneDrive\Documents\Keys\eliteKey.txt') as content:
    elite_key = content.read()

In [3]:
#import nhl_data
nhl_data = pd.read_csv(r'C:\Users\akobe\OneDrive\Desktop\Lighthouse\After\Predicting_Success_all_Rounds_NHL\Data\draft_data_all_rounds.csv', index_col = [0]) #index col makes sures another index column is not needed

In [4]:
nhl_data.head(2)

,draft_year,draft_round,pick_no,team,prospect_name,prospect_id,dob,birth_country,height,weight,shoots,nhl_id,position,prospect_category,amateur_league,amateur_team
0,2009,1,1,New York Islanders,John Tavares,24111.0,1990-09-20,CAN,"6' 1""",209.0,L,8475166.0,C,North American Skater,OHL,London
1,2009,1,2,Tampa Bay Lightning,Victor Hedman,33799.0,1990-12-18,SWE,"6' 6""",220.0,L,8475167.0,D,European Skater,SWEDEN,Modo


In [5]:
#the columns that have missing values 
cols_missing_vals = ['prospect_name', 'prospect_id', 'dob', 'birth_country', 'height', 'weight', 'shoots', 'nhl_id', 'position', 'prospect_category', 
                    'amateur_league', 'amateur_team']

In [6]:
#replacing the missing values 
nhl_data[cols_missing_vals] = nhl_data[cols_missing_vals].replace(0, np.NaN) # replace 0's with NaNs

In [7]:
#drop rows with NaN's 
nhl_data = nhl_data.dropna(subset=['prospect_name', 'prospect_id', 'dob', 'birth_country', 'height', 'weight', 'shoots', 'nhl_id', 'position', 'prospect_category', 
                                'amateur_league', 'amateur_team'])

In [7]:
#update to the correct dob 
#nhl_data.at[11, 'dob'] = '1991-05-09'

In [18]:
nhl_data.to_csv('NaNs_removed.csv')

In [8]:
nhl_data['player_fname'] = nhl_data['prospect_name'].apply(lambda x: x[0:x.find(" ")])

In [9]:
nhl_data['player_lname'] = nhl_data["prospect_name"].apply(lambda x: x[x.find(" ")+1:])

In [10]:
#creates a list of the NHL id's so we can add it to the dataframe to easily join the dataframes afterwards 
nhl_id = []
nhl_id = nhl_data['nhl_id'].tolist()

In [11]:
draft_year = []
draft_year = nhl_data['draft_year'].tolist()

In [12]:
#create a list of player names - for API call 
player_fname = []
player_lname = []
dob = []

player_fname = nhl_data['player_fname'].tolist()
player_lname = nhl_data['player_lname'].tolist()
dob =  nhl_data['dob'].tolist()

In [13]:
#name updated so that can pull API
player_lname[11] = 'De%20Haan'
player_lname[469] = 'St.%20Croix'
player_lname[592] = 'Di%20Giuseppe'
player_lname[643] = 'Di%20Pauli'
player_lname[701] = 'De%20Haas'
player_lname[767] = 'De%20La%20Rose'
player_lname[853] = 'Crus%20Rydberg'
player_lname[908] = 'De%20Jong'
player_lname[924] = 'Dal%20Colle'
player_lname[1005] = 'De%20Leo'
player_lname[1070] = 'Ollas%20Mattsson'
player_lname[1119] = 'Eriksson%20Ek'
player_lname[1142] = 'Forsbacka%20Karlsson'
player_lname[1173] = 'Gates%20Jr.'
player_lname[1175] = 'Vande%20Sompel'
player_lname[1381] = 'Haman%20Aktell'
player_lname[1472] = 'Krag%20Christensen'
player_lname[1623] = 'De%20Jong'
player_lname[1645] = 'Walli%20Walterholm'
player_lname[1666] = 'Roykas%20Marthinsen'
player_lname[1774] = 'St.%20Ivany'
player_lname[1778] = 'Lindstrand%20Cronholm'
player_lname[1818] = 'Emilio%20Pettersen'
player_lname[1834] = 'Worge%20Kreu'

In [14]:
#update to proper dob
dob[11] = '1991-05-09'

In [15]:
player_lname[665]

'Toninato'

In [16]:
#getting all the url's to pull the data from elite prospects 
all_elite_url = []
for i in range(1855):
    first_name = player_fname[i]
    last_name = player_lname[i]
    birth_date = dob[i]
    elite_url = 'https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name='+first_name+'%20'+last_name+'&dateOfBirth='+str(birth_date)+'&apiKey='
    all_elite_url.append(elite_url)

In [28]:
all_elite_url[665:1000]

['https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name=Dominic%20Toninato&dateOfBirth=1994-07-04&apiKey=',
 'https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name=Brian%20Cooper&dateOfBirth=1994-03-09&apiKey=',
 'https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name=Daniel%20Gunnarsson&dateOfBirth=1993-11-01&apiKey=',
 'https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name=Seth%20Griffith&dateOfBirth=1993-05-19&apiKey=',
 'https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name=Michael%20Clarke&dateOfBirth=1993-01-04&apiKey=',
 'https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name=Logan%20Nelson&dateOfBirth=1994-04-29&apiKey=',
 'https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name=Branden%20Troock&dateOfBirth=1993-09-09&apiKey=',
 'https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name=Graham%20Black&dat

In [18]:
#all_elite_url_1 = all_elite_url[0:500]
#all_elite_url_2 = all_elite_url[501:1000]
#all_elite_url_3 = all_elite_url[1001:1500]
#all_elite_url_4 = all_elite_url[1501:]
all_elite_url_2_2 = all_elite_url[665:1000]

In [117]:
#elite_data_worked = elite_data[0:500]

In [118]:
#elite_data999 = elite_data

In [119]:
#elite1498 = elite_data

In [19]:
len(all_elite_url_2_2)

335

In [116]:
#elite_data = elite_data_worked

In [27]:
elite_data= []

#### split into 4 as errorred out when trying to pull everything at once 

In [70]:
#get request for all url's 
for j in range(len(all_elite_url_1)):
    response = requests.get(all_elite_url_1[j]+elite_key)
    elite_data.append(response.json())

In [23]:
#get request for url's 665-1000
for j in range(len(all_elite_url_2_2)):
    response = requests.get(all_elite_url_2_2[j]+elite_key)
    elite_data.append(response.json())

In [83]:
#get request for all url's 
for j in range(len(all_elite_url_3)):
    response = requests.get(all_elite_url_3[j]+elite_key)
    elite_data.append(response.json())

In [87]:
#get request for all url's 
for j in range(len(all_elite_url_4)):
    response = requests.get(all_elite_url_4[j]+elite_key)
    elite_data.append(response.json())

In [33]:
len(elite_data)

335

In [35]:
tester = []

In [36]:
response = requests.get('https://api.eliteprospects.com/v1/players?offset=0&limit=1000&sort=name&name=Blake%20Heinrich&dateOfBirth=1995-01-30&apiKey='+elite_key)
tester.append(response.json())

<IPython.core.display.JSON object>

In [24]:
from IPython.display import JSON
JSON(elite_data)

<IPython.core.display.JSON object>

In [100]:
#creating empty lists 
elite_id = []
elite_name = []

In [101]:
for i in range(664):
    if len(elite_data[i]['data']) > 0:
        elite_id.append(elite_data[i]['data'][0]['id'])
        elite_name.append(elite_data[i]['data'][0]['name'])
    else:
        #if there is no value replace with NAN
        elite_id.append('NaN')
        elite_name.append('NaN')  

KeyError: 0

In [94]:
#create a data frame with the information 
elite_data = pd.DataFrame(list(zip(nhl_id, elite_id, elite_name, dob)),
               columns =['nhl_id', 'elite_id', 'elite_name', 'elite_dob'])

In [96]:
elite_data.head(2)

,nhl_id,elite_id,elite_name,elite_dob
0,8475166.0,9223,John Tavares,1990-09-20
1,8475167.0,6007,Victor Hedman,1990-12-18


In [97]:
#find NaN's 
elite_data.isnull().sum()

nhl_id        0
elite_id      0
elite_name    0
elite_dob     0
dtype: int64

In [98]:
#save the data to csv - 0 - 663 (Joey LaLeggia)
#elite_data.to_csv('info_for_pulling_prospects1.csv')

In [ ]:
#save the data to csv -
elite_data.to_csv('info_for_pulling_prospects2.csv')

# draft year data 

In [32]:
elite_data['draft_year'] = draft_year

In [35]:
#draft season is the year before and year of draft
#this information is needed for the API call to only get stats for their draft year 
draft_season = []

for i in elite_data['draft_year']: 
    if i == 2009:
        draft_season.append('2008-2009')
    elif i == 2010:
        draft_season.append('2009-2010')
    elif i == 2011:
        draft_season.append('2010-2011')
    elif i == 2012:
        draft_season.append('2011-2012')
    elif i == 2013:
        draft_season.append('2012-2013')
    elif i == 2014:
        draft_season.append('2013-2014')
    elif i == 2015:
        draft_season.append('2014-2015')
    elif i == 2016:
        draft_season.append('2015-2016')
    elif i == 2017:
        draft_season.append('2016-2017')
    elif i == 2018:
        draft_season.append('2017-2018')

In [36]:
#add the list to the dataframe 
elite_data['draft_season'] = draft_season

In [38]:
elite_id = []
draft_season = []

elite_id = elite_data['elite_id'].tolist()
draft_season = elite_data['draft_season'].tolist()

## Pull Prospect stats

In [45]:
elite_data = pd.read_csv(r'C:\Users\akobe\lighthouse-data-notes\Final-Data\info_for_pulling_prospects.csv', index_col = [0]) #index col makes sures another index column is not needed

In [50]:
#urls for draft year stats 
all_elite_prospect_url = []
for i in range(len(elite_data)):
    ident = elite_id[i]
    season = draft_season[i]
    elite_prospect_url = 'https://api.eliteprospects.com/v1/players/'+str(ident)+'/stats?offset=0&limit=100&sort=season&season='+season+'&team.teamType=club&apiKey='
    all_elite_prospect_url.append(elite_prospect_url)

In [51]:
elite_prospect_data = []

for j in range(296):
    response = requests.get(all_elite_prospect_url[j]+elite_key)
    elite_prospect_data.append(response.json())

In [52]:
from IPython.display import JSON
JSON(elite_prospect_data)

<IPython.core.display.JSON object>

In [54]:
#replaces null values with 0's 
elite_prospect_data_no_null = json.loads(json.dumps(elite_prospect_data).replace('null', '0'))

In [55]:
JSON(elite_prospect_data_no_null)

<IPython.core.display.JSON object>

In [56]:
#creating empty lists for the data 
prospect_gp = []
prospect_g = []
prospect_a = []
prospect_pts = []
prospect_pim = []
prospect_pm = []

In [57]:
# creates lists of all variables wanted, if a player was with more than one team in their draft year their stats are added together 
##this one mostly works

for prospect in elite_prospect_data_no_null[:]:
    gp = 0
    goals = 0
    assists = 0
    pts = 0
    pim = 0
    pm = 0
    
    prospect_data = prospect['data']
    i = 0
    for prospect_stats in prospect_data:
        #some players were with more than one team in their draft year, this goes through each team and adds the data together 
        i+=1
        if prospect_stats['regularStats'] == 0:   #if there is no info at regularStats, this spasses it and moves onto the next 
            pass
        else:   
            gp += prospect_stats['regularStats']['GP'] 
            goals += prospect_stats['regularStats']['G']
            assists += prospect_stats['regularStats']['A']
            pts += prospect_stats['regularStats']['PTS']
            pim += prospect_stats['regularStats']['PIM']
            pm +=  prospect_stats['regularStats']['PM']

    #appends the information to a new list         
    prospect_gp.append(gp)  
    prospect_g.append(goals)  
    prospect_a.append(assists)  
    prospect_pts.append(pts)  
    prospect_pim.append(pim)  
    prospect_pm.append(pm)  

In [58]:
#create a data frame with the information 
prospect_data = pd.DataFrame(list(zip(elite_id, prospect_gp, prospect_g, prospect_a, prospect_pts, prospect_pim, prospect_pm)),
               columns =['elite_id', 'prospect_gp', 'prospect_g', 'prospect_a', 'prospect_pts', 'prospect_pim', 'prospect_pm'])              

In [157]:
#save the data to csv 
#prospect_data.to_csv('prospect_data.csv')

## Joining elite dataframe 

In [60]:
#joining tables 
all_elite_prospect_data = pd.merge(elite_data, prospect_data,  on = 'elite_id')
all_elite_prospect_data.head()

,nhl_id,elite_id,elite_name,elite_dob,draft_year,draft_season,prospect_gp,prospect_g,prospect_a,prospect_pts,prospect_pim,prospect_pm
0,8475166,9223,John Tavares,1990-09-20,2009,2008-2009,56,58,46,104,54,10
1,8475167,6007,Victor Hedman,1990-12-18,2009,2008-2009,45,7,16,23,62,21
2,8475168,12571,Matt Duchene,1991-01-16,2009,2008-2009,57,31,48,79,42,32
3,8475169,14657,Evander Kane,1991-08-02,2009,2008-2009,61,48,48,96,89,51
4,8475170,14739,Brayden Schenn,1991-08-22,2009,2008-2009,70,32,56,88,82,20


In [161]:
#save the data to csv 
#all_elite_prospect_data.to_csv('all_elite_prospect_data.csv')